In [0]:
from e3nn import o3

irreps = o3.Irreps("3x0e + 2x1o")
import torch
alpha, beta, gama = torch.rand(3)

In [4]:
irreps.D_from_angles(alpha, beta, gama)

tensor([[ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.4124,  0.0723,  0.9081,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.1184,  0.9841, -0.1321,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.9033,  0.1620,  0.3973,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.4124,  0.0723,
          0.9081],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.1184,  0.9841,
         -0.1321],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.9033,  0.1620,
          0.3973]])

In [8]:
from e3nn import o3 #rotation and parity
import torch

#vector *vector

tp = o3.FullTensorProduct("1o", "1o")
tp(torch.tensor([1.0, 2.0, 3.0]), torch.tensor([4.0, 5.0, 6.0]))

tensor([18.4752, -2.1213,  4.2426, -2.1213, 12.7279,  9.1924, -0.8165, 19.0919,
         9.8995])

In [2]:
import torch
from e3nn import o3
from e3nn.nn import FullyConnectedNet, Gate
from e3nn.o3 import Irreps

class NBodyModel(torch.nn.Module):
    def __init__(self, n_bodies):
        super().__init__()

        irreps_in = Irreps(f"{n_bodies}x0e + {n_bodies}x1o")
        irreps_hidden = Irreps("16x0e + 16x1o") 
        irreps_out = Irreps(f"{n_bodies}x1o")

        self.layers = torch.nn.Sequential(
            FullyConnectedNet([irreps_in.dim, 128, irreps_hidden.dim], torch.relu),
            Gate(
                irreps_scalars="16x0e",
                act_scalars=[torch.relu],
                irreps_gates="16x0e",
                act_gates=[torch.sigmoid],
                irreps_gated="16x1o"
            ),
            FullyConnectedNet([irreps_hidden.dim, irreps_out.dim], torch.relu)
        )

    def forward(self, x):
        return self.layers(x)


learning_rate = 1e-4
epochs = 100
batch_size = 32

#n_bodies = data["positions"].shape[1]
n_bodies = 5
model = NBodyModel(n_bodies)

/home/mightycarrot/anaconda3/envs/n_body_approx/lib/python3.8/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    for batch in DataLoader(train_data, batch_size=batch_size, shuffle=True):
        positions, masses, forces = batch['positions'], batch['masses'], batch['forces']

        # Combine positions and masses as input
        inputs = torch.cat((positions, masses.unsqueeze(-1)), dim=-1)

        # Model predictions
        predictions = model(inputs)

        # Compute loss
        loss = torch.nn.functional.mse_loss(predictions, forces)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    test_positions, test_masses, test_forces = test_data['positions'], test_data['masses'], test_data['forces']
    inputs = torch.cat((test_positions, test_masses.unsqueeze(-1)), dim=-1)
    test_predictions = model(inputs)
    test_loss = torch.nn.functional.mse_loss(test_predictions, test_forces)
print(f"Test Loss: {test_loss.item()}")
